In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import binom
from ipywidgets import interact, interactive, fixed, IntSlider, FloatSlider, Layout, HBox, VBox, HTML, Label, BoundedIntText
from IPython.display import display, clear_output

# --- 1. Define the core calculation and plotting function ---
def plot_binomial_explorer(n, p, k_calc):
    """
    Calculates binomial probabilities, plots the PMF, and displays the calculator results.
    This function is called by the ipywidgets `interact`.
    """
    
    # Clear previous output for a cleaner update (important in Jupyter)
    clear_output(wait=True)
    
    # --- CALCULATIONS ---
    
    # Ensure k is within bounds (although BoundedIntText widget handles this)
    if k_calc < 0 or k_calc > n:
        display(HTML(f'<p style="color:red;">Error: k must be between 0 and {n}.</p>'))
        return
        
    prob_eq_k = binom.pmf(k_calc, n, p)
    prob_le_k = binom.cdf(k_calc, n, p)
    
    # P(X >= k) = 1 - P(X <= k-1)
    if k_calc == 0:
        prob_ge_k = 1.0
    else:
        prob_ge_k = 1.0 - binom.cdf(k_calc - 1, n, p)

    # --- PLOTTING ---
    k_vals = np.arange(n + 1)
    pmf_vals = binom.pmf(k_vals, n, p)
    mean_val = n * p

    fig, ax = plt.subplots(figsize=(10, 5))
    ax.bar(k_vals, pmf_vals, color='skyblue', edgecolor='darkblue', alpha=0.8)

    # Mean line
    ax.axvline(mean_val, color='red', linestyle='--', linewidth=1.5, label=f'Mean = {mean_val:.2f}')

    # Highlight k value
    ax.bar(k_calc, binom.pmf(k_calc, n, p), color='orange', edgecolor='darkred', linewidth=1, label=f'k = {k_calc}', alpha=0.9)

    ax.set_title(f"Binomial PMF: B(n={n}, p={p})", fontsize=16, fontweight='bold')
    ax.set_xlabel("Number of Successes (k)", fontsize=12)
    ax.set_ylabel("P(X = k)", fontsize=12)
    ax.set_xticks(np.unique([0, n//2, n]))
    if n <= 20: # Show all ticks for small n
        ax.set_xticks(k_vals)
    ax.grid(axis='y', linestyle=':', alpha=0.6)
    ax.legend()
    
    plt.show() # Display the Matplotlib plot

    # --- DISPLAY CALCULATOR RESULTS ---
    
    html_output = f"""
    <div style="font-family: sans-serif; padding-top: 10px;">
        <h3 style="margin-bottom: 5px;">Binomial Calculator Results (k = {k_calc})</h3>
        <table style="width: 100%; border-collapse: collapse;">
            <tr>
                <td style="padding: 8px; border: 1px solid #ddd; background-color: #f2f2f2;">
                    <strong>P(X = {k_calc})</strong>
                </td>
                <td style="padding: 8px; border: 1px solid #ddd;">
                    {prob_eq_k:.6f} ({prob_eq_k*100:.4f}%)
                </td>
            </tr>
            <tr>
                <td style="padding: 8px; border: 1px solid #ddd; background-color: #f2f2f2;">
                    <strong>P(X &le; {k_calc})</strong>
                </td>
                <td style="padding: 8px; border: 1px solid #ddd;">
                    {prob_le_k:.6f} ({prob_le_k*100:.4f}%)
                </td>
            </tr>
            <tr>
                <td style="padding: 8px; border: 1px solid #ddd; background-color: #f2f2f2;">
                    <strong>P(X &ge; {k_calc})</strong>
                </td>
                <td style="padding: 8px; border: 1px solid #ddd;">
                    {prob_ge_k:.6f} ({prob_ge_k*100:.4f}%)
                </td>
            </tr>
        </table>
    </div>
    """
    display(HTML(html_output))


# --- 2. Define the Widgets (Inputs) ---

# Binomial Parameters
n_slider = IntSlider(min=1, max=100, step=1, value=20, 
                     description='Trials (n):', style={'description_width': 'initial'})
p_slider = FloatSlider(min=0.01, max=0.99, step=0.01, value=0.5, 
                       description='Probability (p):', style={'description_width': 'initial'},
                       continuous_update=False) # Only update on release

# Binomial Calculator k
# Use BoundedIntText for k, but its max needs to be dynamically linked to n.
# For simplicity in 'interact', we use IntSlider and trust the main function validation.
k_calc_slider = IntSlider(min=0, max=100, step=1, value=10, 
                          description='k (0 ≤ k ≤ n):', style={'description_width': 'initial'})


# --- 3. Assemble and Run the Interactive Interface ---

# Use interactive() to link widgets to the function
interactive_plot = interactive(
    plot_binomial_explorer,
    n=n_slider,
    p=p_slider,
    k_calc=k_calc_slider
)

# Arrange the widgets in a control panel
controls = VBox([
    HTML("<h3>Binomial Distribution Parameters</h3>"),
    n_slider, 
    p_slider,
    HTML("<hr><h3>Binomial Calculator Input</h3>"),
    k_calc_slider
], layout=Layout(width='300px', padding='10px', border='1px solid lightgray'))

# Display the controls next to the output plot
# The output plot is handled by interactive_plot.children[-1]
# We use HBox to put controls and output side-by-side
full_app = HBox([controls, interactive_plot.children[-1]])

# Display the main application
display(HTML("<h2>Binomial PMF & Calculator (Jupyter/IPyWidgets)</h2>"))
display(full_app)


HTML(value='<h2>Binomial PMF & Calculator (Jupyter/IPyWidgets)</h2>')